In [ ]:
# Step 1: Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [ ]:
# Step 2: Load dataset
df = pd.read_csv("creditcard.csv")
print("Dataset shape:", df.shape)


Dataset shape: (284807, 31)


In [ ]:
# Step 3: Split features and labels
X = df.drop("Class", axis=1).values
y = df["Class"].values
print(X.shape)
print(y.shape)


(284807, 30)
(284807,)


In [ ]:
# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(X_train.shape, X_test.shape)


(227845, 30) (56962, 30)


In [ ]:
# Step 5: Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Step 6: Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [ ]:
# Step 7: Define the neural network
class FraudNet(nn.Module):
    def __init__(self, input_dim):
        super(FraudNet, self).__init__()
        self.layer1 = nn.Linear(input_dim, 32)
        self.layer2 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.output(x))
        return x

In [ ]:
# Step 8: Initialize model, loss, optimizer
model = FraudNet(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Step 9: Train model
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 2/10, Loss: 0.7818
Epoch 4/10, Loss: 0.7721
Epoch 6/10, Loss: 0.7623
Epoch 8/10, Loss: 0.7525
Epoch 10/10, Loss: 0.7425


In [ ]:
# Step 10: Evaluate model
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = (y_pred >= 0.5).float()
    accuracy = (y_pred_cls.eq(y_test).sum() / y_test.shape[0]).item()
    print(f"\nTest Accuracy: {accuracy:.4f}")


Test Accuracy: 0.0738
